In [1]:

import sys

sys.path.append('../')
%load_ext autoreload
%autoreload 2

from models.data_process import *
from models.run_MLP import MLP
from anchor.anchor_tabular import *

args = parse_args()
device = args.device

In [2]:
model = MLP().to(device)
ckpt = torch.load('../' + args.model_path + f'MPL_{args.epoch}.pth', map_location='cpu')
model.load_state_dict(ckpt)
model.eval()
# model = load_model()

MLP(
  (net): Sequential(
    (0): Linear(in_features=102, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

## Encoder

In [3]:
x_dataset, target, encoder, categorical_names = load_adult_income_dataset(baseDir='../')
print(x_dataset.shape)
sample_id = 0
x_test = x_dataset[sample_id]
print(x_test)

(32561, 12)
[0 39 13 40 6 9 4 0 1 4 1 38]


In [4]:
predict_fn = lambda x: model.predict_anchor(x, encoder)
adult_process_names.insert(0, 'id')
explainer = AnchorTabularExplainer(adult_target_value, adult_process_names, x_dataset,
                                   categorical_names=categorical_names)
exp = explainer.explain_instance(x_test, predict_fn, threshold=0.95)

features = []
for f in exp.features():
    features.append(adult_process_names[f])
print(f'Anchor sample_id {sample_id}: ', features)
print(exp.features())


[4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3]
covered_false_idx [5403 19928 11018 26533 18382 18795 31911 18541 15493 26115]
covered_false_idx [3387 14870 28929 2727 30279 4220 18089 32374 18382 28213]
covered_false_idx [9929 23166 17899 5246 22638 17106 2895 14619 16536 26649]
[2, 4, 3]


In [5]:
ret = explainer.to_explanation_map(exp.exp_map)
# print(ret['labelNames'])
# print(ret['predictProba'])
explanation = ret['explanation']
print("raw_data: ", ret['rawData'])
print("explanation--names: ", explanation['names'])
# print("explanation--certainties: ", explanation['certainties'])
# print("explanation--examples: ", explanation['examples'][0])
# print("explanation--examples: ", explanation['examples'][1])
# print("explanation--keys: ", explanation['examples'][0].keys())
# print("explanation--coveredFalse: ", explanation['examples'][0]['covered_true_idx'])
# print("exp.exp_map: ",exp.exp_map['examples'][0]['covered_true'])

# features_in_anchor = set(exp.exp_map['feature'][:1])
# ret_obj = explainer.transform_to_examples(exp.exp_map['examples'][0]['covered_false'], features_in_anchor,
#                                           exp.exp_map['prediction'])
# print(ret_obj)
# [21644 7914 18320 17622 4782 12837 13096 3721 799 10951]

[32514 12776 6758 28980 29773 23868 25633 8047 11624 1453]
[5403 19928 11018 26533 18382 18795 31911 18541 15493 26115]
[42 18599 8012 4346 9556 15353 5259 713 9133 13397]
[12430 11724 19617 7269 11726 14367 28792 8266 14367 27380]
[3387 14870 28929 2727 30279 4220 18089 32374 18382 28213]
[24838 30416 18117 8712 8928 9478 4616 29372 11357 26603]
[9929 23166 17899 5246 22638 17106 2895 14619 16536 26649]
[19662 10598 31944 19906 11711 5285 5686 13446 18746 4922]
instance [0 39 13 40 6 9 4 0 1 4 1 38]
raw_data:  [('id', 'id=0', -1), ('age', '37.00 < age <= 48.00', -1), ('educational-num', 'educational-num > 12.00', -1), ('hours-per-week', 'hours-per-week <= 40.00', -1), ('workclass', 'State-gov', -1), ('education', 'Bachelors', -1), ('marital-status', 'Never-married', -1), ('occupation', 'Adm-clerical', -1), ('relationship', 'Not-in-family', -1), ('race', 'White', -1), ('gender', 'Male', -1), ('native-country', 'United-States', -1)]
explanation--names:  ['marital-status', 'relationship'

In [6]:
exp.show_in_notebook()

[32514 12776 6758 28980 29773 23868 25633 8047 11624 1453]
[5403 19928 11018 26533 18382 18795 31911 18541 15493 26115]
[42 18599 8012 4346 9556 15353 5259 713 9133 13397]
[12430 11724 19617 7269 11726 14367 28792 8266 14367 27380]
[3387 14870 28929 2727 30279 4220 18089 32374 18382 28213]
[24838 30416 18117 8712 8928 9478 4616 29372 11357 26603]
[9929 23166 17899 5246 22638 17106 2895 14619 16536 26649]
[19662 10598 31944 19906 11711 5285 5686 13446 18746 4922]
instance [0 39 13 40 6 9 4 0 1 4 1 38]


In [7]:
print(categorical_names)

{4: array(['Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'],
      dtype=object), 5: array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
       'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
       'Masters', 'Preschool', 'Prof-school', 'Some-college'],
      dtype=object), 6: array(['Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'],
      dtype=object), 7: array(['Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial',
       'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct',
       'Other-service', 'Priv-house-serv', 'Prof-specialty',
       'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'],
      dtype=object), 8: array(['Husband', 'Not-in-family', 'Other-relative', 'Own-child',
       'Unmarried', 'Wife'], dtype=object), 9: array(['Amer-Indian-Eskimo', 'Asian-P

In [8]:
print(explainer.categorical_features)

[4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3]
